# 3.2.3 Multiple Regression From Simple Univariate Regression

Suppose we have a *univariate* (p = 1) model with no intercept (3.23):
$$Y=X\beta+\varepsilon$$

The least squares estimate and residuals are (3.24):
$$
\hat{\beta} = \cfrac{\sum_1^N {x_iy_i}}{\sum_1^N {x_i^2}} \\
r_i = y_i - x_i\hat{\beta}
$$

With the inner product:
$$
\hat{\beta} = \cfrac{\langle \mathbf{x}, \mathbf{y} \rangle}{\langle \mathbf{x}, \mathbf{x}\rangle}\\
\mathbf{r} = \mathbf{y} - \mathbf{x}\hat{\beta}
$$

Suppose that the columns of the matrix **X** are orthogonal; that is $\langle \mathbf{x}_j, \mathbf{x}_k \rangle = 0$
then it is easy to check that $\hat{\beta_j} = \langle \mathbf{x}_j, \mathbf{y} \rangle / \langle \mathbf{x}_j, \mathbf{x}_j \rangle$, i.e the inputs have no effect on each other's parameter estimates.

Suppose next that we have an intercept and a single input x (3.27):
$$\hat{B}_1 = \cfrac{\langle \mathbf{x} - \overline{x}\mathbf{1}, \mathbf{y} \rangle}{ \langle \mathbf{x} - \overline{x}\mathbf{1}, \mathbf{x} - \overline{x}\mathbf{1} \rangle}$$

We can view the estimate as the result of two simple regression:

1. Regress **x** on 1 to produce the residual $\mathbf{z} = \mathbf{x} - \overline{x}\mathbf{1}$

2. Regress **y** on the residual **z** to give the coefficient $\hat{\beta}_1$.

Regress **b** on **a** means $\hat{\gamma}=\langle \mathbf{a},\mathbf{b} \rangle / \langle \mathbf{a}, \mathbf{a}\rangle$ and the residual vector $\mathbf{b} - \hat{\gamma}\mathbf{a}$.

This recipe generalizes to the case of *p* inputs, as shown in Algorithm 3.1.

**Algorithm 3.1 Regression by Successive Orthogonalization**
1. $\mathbf{z}_0 = \mathbf{x}_0 = \mathbf{1}$

2. For $j = 1, 2, \cdots, p$
   
   * Regress $\mathbf{x}_j$ on $\mathbf{z}_0,...,\mathbf{z}_{j - 1}$ to produce $\hat{\gamma}_{lj}=\langle \mathbf{z}_l, \mathbf{x}_j \rangle / \langle \mathbf{z}_l,\mathbf{z}_l \rangle$ $l=0,\cdots,j-1$, and residualt vector $\mathbf{z}_j=\mathbf{x}_j - \sum_{k=0}^{j-1} \hat{\gamma}_{kj}\mathbf{z}_k$

3. Regress $\mathbf{y}$ on the residual $\mathbf{z}_p$ to give the estimate $\hat{\beta}_p$

The result of this algorithm is (3.28):

$$\hat{\beta}_p=\cfrac{\langle \mathbf{z}_p, \mathbf{y} \rangle}{\langle \mathbf{z}_p,\mathbf{z}_p \rangle}$$

If $\mathbf{x}_p$ is highly correlated with some of the other $\mathbf{x}_k$'s the residual vector $\mathbf{x}_p$ will be close to zero, and from (3.28) the coefficient $\hat{\beta}_p$ will be unstable. 

From (3.28) we also obtain an alternative formula for the variance estimates (3.8):

$$Var(\hat{\beta}_p) = \cfrac{\sigma^2}{\langle \mathbf{z}_p, \mathbf{z}_p \rangle}=\cfrac{\sigma^2}{||\mathbf{z}_p||^2}  $$

On other words, the precision with which we can estimate $\hat{\beta}_p$ depends on the lengths of the residual vector $\mathbf{z}_p$;